In [1]:
from xml.dom import minidom
import pandas as pd
import numpy as np
import os
import glob
import time
from datetime import datetime
from datetime import timedelta
import gzip
import mechanicalsoup
import urllib
import requests
from bs4 import BeautifulSoup
import json
import time
from langdetect import detect

In [2]:
def get_scopus_data(index, file):
    soup = BeautifulSoup(open(file, encoding='utf-8'), 'html.parser')
    
    arr = soup.find_all('p')
    abstract = str(arr[1])
    abstract = abstract[abstract.find('>') + 1:-4]
    available = 'Articles not published yet, but available online' not in abstract
    
    arr = soup.find_all('ul')
    lines = arr[4].find_all('li')
    
    authors = []
    for line in lines:
        line = str(line)
        if(len(line[line.find('\">')+2:line.find('.')+1])):
            authors.append(line[line.find('\">')+2:line.find('.')+1])

    arr = soup.find_all('h2')
    title = str(arr)
    i = title.find('\">')
    title = title[i + 2:title.find('<', i)]
    
    language = detect(abstract)
    
    data = {}
    data['id'] = index
    data['title'] = title
    data['authors'] = authors
    data['abstract'] = abstract
    data['language'] = language
    json_data = json.dumps(data)
    return json_data, available

def get_newest_file(path):
    files = os.listdir(path)
    paths = [os.path.join(path, basename) for basename in files]
    return max(paths, key=os.path.getctime)

In [3]:
xmldoc = minidom.parse('downloads\\cobiss-org\\2019-07-18-145628_cobiss.xml')
itemlist = xmldoc.getElementsByTagName('Scopus')
links = []
for s in itemlist:
    if len(s.childNodes) != 0:
        links.append(s.childNodes[0].nodeValue)
        
links;

In [4]:
articleIds = []

for link in links:
    articleIds.append(link[link.rfind('-')+1:])

In [5]:
now = datetime.now()

for index in articleIds:
    if not os.path.exists('downloads/scopus/{:}'.format(index)):
        os.mkdir('downloads/scopus/{:}'.format(index))
        
    if len(os.listdir('downloads/scopus/{:}'.format(index))) != 0:
        file = get_newest_file('downloads/scopus/{:}'.format(index))
        file_date = datetime.strptime(file[file.find('\\')+1:-5], '%Y-%m-%d')
        if file_date < now - timedelta(days = 14):
            url = 'http://www.scopus.com/inward/record.url?partnerID=2dRBettD&eid=2-s2.0-{:}'.format(index)
            response = requests.get(url)

            file = open('downloads/scopus/{:}/{:}.html'.format(index, now.strftime('%Y-%m-%d')), "w", encoding='utf-8')
            file.write(response.text)
            file.close()
    else:
        url = 'http://www.scopus.com/inward/record.url?partnerID=2dRBettD&eid=2-s2.0-{:}'.format(index)
        response = requests.get(url)

        file = open('downloads/scopus/{:}/{:}.html'.format(index, now.strftime('%Y-%m-%d')), "w", encoding='utf-8')
        file.write(response.text)
        file.close()

In [6]:
directories = os.listdir('./downloads/scopus')
file_data = []

for directory in directories:
    file = get_newest_file('./downloads/scopus/{:}'.format(directory))
    
    article_data, available = get_scopus_data(directory, file)
    if available:
        file_data.append(article_data)
        
    
file_data;

In [7]:
tstamp = datetime.now().strftime('%Y-%m-%d-%H%M%S')
file_name = 'data\\scopus_data_{:}.json'.format(tstamp)
with open(file_name, 'w') as f:
    f.write('[\n')
    for item in file_data:
        if (item == file_data[len(file_data) - 1]):
            f.write("%s\n" % item)
        else:
            f.write("%s,\n" % item)
    f.write(']')